In [1]:
%%capture
!pip install sentence-transformers
!pip install -U datasets

### Load In Model

In [2]:
from sentence_transformers import SentenceTransformer, models

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
model

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

### Load In Dataset

In [3]:
from datasets import load_dataset

dataset_id = "embedding-data/QQP_triplets"

dataset = load_dataset(dataset_id)

README.md: 0.00B [00:00, ?B/s]

quora_duplicate_triplets.jsonl:   0%|          | 0.00/183M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/101762 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['set'],
        num_rows: 101762
    })
})

In [5]:
print(f"- The {dataset_id} dataset has {dataset['train'].num_rows} examples.")
print(f"- Each example is a {type(dataset['train'][0])} with a {type(dataset['train'][0]['set'])} as value.")
print(f"- Examples look like this: {dataset['train'][0]}")

- The embedding-data/QQP_triplets dataset has 101762 examples.
- Each example is a <class 'dict'> with a <class 'dict'> as value.
- Examples look like this: {'set': {'query': 'Why in India do we not have one on one political debate as in USA?', 'pos': ['Why cant we have a public debate between politicians in India like the one in US?'], 'neg': ['Can people on Quora stop India Pakistan debate? We are sick and tired seeing this everyday in bulk?', 'Why do politicians, instead of having a decent debate on issues going in and around the world, end up fighting always?', 'Can educated politicians make a difference in India?', 'What are some unusual aspects about politics and government in India?', 'What is debate?', 'Why does civic public communication and discourse seem so hollow in modern India?', 'What is a Parliamentary debate?', "Why do we always have two candidates at the U.S. presidential debate. yet the ballot has about 7 candidates? Isn't that a misrepresentation of democracy?", 'Wh

## Contrastive Loss

### Prepare Dataset for Training

In [6]:
from datasets import Dataset

train_data = dataset['train']
n_examples = len(train_data) // 100

anchors, positives, negatives = [], [], []

for i in range(n_examples):
    example_set = train_data[i]['set']

    anchor = f"{example_set['query']}"
    positive = f"{example_set['pos'][0]}"
    negative = f"{example_set['neg'][0]}"

    anchors.append(anchor)
    positives.append(positive)
    negatives.append(negative)

train_dataset = Dataset.from_dict({
    "anchor": anchors,
    "positive": positives,
    "negative": negatives,
})


# --- Test dataset (new, 100 examples) ---
anchors_test, positives_test, negatives_test = [], [], []
for i in range(n_examples, n_examples+100):
    example_set = train_data[i]['set']
    anchors_test.append(f"{example_set['query']}")
    positives_test.append(f"{example_set['pos'][0]}")
    negatives_test.append(f"{example_set['neg'][0]}")

test_dataset = Dataset.from_dict({
    "anchor": anchors_test,
    "positive": positives_test,
    "negative": negatives_test,
})

In [7]:
print(f"We have a {type(train_dataset)} of length {len(anchors)} containing {type(anchors[0])}'s.")
print(f"We have a {type(test_dataset)} of length {len(anchors_test)} containing {type(anchors_test[0])}'s.")

We have a <class 'datasets.arrow_dataset.Dataset'> of length 1017 containing <class 'str'>'s.
We have a <class 'datasets.arrow_dataset.Dataset'> of length 100 containing <class 'str'>'s.


### Test Model Prior to Training

In [8]:
import pandas as pd
import numpy as np

def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


anchor_embs = model.encode(anchors_test, convert_to_numpy=True)
positive_embs = model.encode(positives_test, convert_to_numpy=True)
negative_embs = model.encode(negatives_test, convert_to_numpy=True)

query_pos_sims = [cosine_sim(a, p) for a, p in zip(anchor_embs, positive_embs)]
query_neg_sims = [cosine_sim(a, n) for a, n in zip(anchor_embs, negative_embs)]

# --- Build DataFrame ---
df = pd.DataFrame({
    "query": anchors_test,
    "pos": positives_test,
    "neg": negatives_test,
    "query_pos_sim": query_pos_sims,
    "query_neg_sim": query_neg_sims,
})

df['pos_neg_diff'] = df['query_pos_sim'] - df['query_neg_sim']
df.head()

,query,pos,neg,query_pos_sim,query_neg_sim,pos_neg_diff
0,"What is the difference in quality between: BD,...",What's the difference between 1080i and 1080p?,What is the difference between 1080p and 2160p...,0.724870,0.630589,0.094281
1,How can I hide my mutuality on Facebook?,How can I hide my mutual friends list on faceb...,How do I hide my likes and comments from my fr...,0.832621,0.630010,0.202611
2,How many clothes canyou make limited or unlimi...,How many clothes can the clothes company make ...,What are clothing stores that sell inexpensive...,0.913175,0.354658,0.558517
3,How is GST beneficial for a common man?,How is GST beneficial for the country? How wou...,What exactly is GST? It's consequences and ben...,0.740855,0.785609,-0.044754
4,What painting/drawing describes your personali...,What painting or drawing describes your person...,How does art influence feelings and make an im...,0.994289,0.632195,0.362094


In [9]:
print(f"Average Pos/Neg Diff: {df['pos_neg_diff'].mean()}")
print(f"Median Pos/Neg Diff: {df['pos_neg_diff'].median()}")

Average Pos/Neg Diff: 0.22368879616260529
Median Pos/Neg Diff: 0.21007347106933594


### Train



In [10]:
from sentence_transformers import losses

train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [11]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

training_args = SentenceTransformerTrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=1, # We use a batch size of 1 because Multiple Negatives Ranking Loss produces in-batch negatives.
    num_train_epochs=3,
    remove_unused_columns=False,

    # 👇 This ensures logging happens every epoch
    logging_strategy="epoch",
    logging_steps=1,  # Still required, but ignored when logging_strategy="epoch"

    # Optional: save checkpoints every epoch too
    save_strategy="epoch",
    save_total_limit=2,
)

trainer = SentenceTransformerTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    loss=train_loss,
)

trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: duypham (duypham-freelancer) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1017,0.089100
2034,0.002600
3051,0.000300


TrainOutput(global_step=3051, training_loss=0.03066799434354289, metrics={'train_runtime': 2095.9148, 'train_samples_per_second': 1.456, 'train_steps_per_second': 1.456, 'total_flos': 0.0, 'train_loss': 0.03066799434354289, 'epoch': 3.0})

### Testing Our Results

As you can see, the model improves it's discriminability between positive and negative pairs in the hold out test dataset by ~2x.

This is a result of fine-tuning to the dataset and showing it positive and negative pairs for each query.

In [12]:
anchor_embs = model.encode(anchors_test, convert_to_numpy=True)
positive_embs = model.encode(positives_test, convert_to_numpy=True)
negative_embs = model.encode(negatives_test, convert_to_numpy=True)

query_pos_sims = [cosine_sim(a, p) for a, p in zip(anchor_embs, positive_embs)]
query_neg_sims = [cosine_sim(a, n) for a, n in zip(anchor_embs, negative_embs)]

# --- Build DataFrame ---
df = pd.DataFrame({
    "query": anchors_test,
    "pos": positives_test,
    "neg": negatives_test,
    "query_pos_sim": query_pos_sims,
    "query_neg_sim": query_neg_sims,
})

df['pos_neg_diff'] = df['query_pos_sim'] - df['query_neg_sim']
df.head()

,query,pos,neg,query_pos_sim,query_neg_sim,pos_neg_diff
0,"What is the difference in quality between: BD,...",What's the difference between 1080i and 1080p?,What is the difference between 1080p and 2160p...,0.684773,0.431788,0.252985
1,How can I hide my mutuality on Facebook?,How can I hide my mutual friends list on faceb...,How do I hide my likes and comments from my fr...,0.823714,0.515045,0.308669
2,How many clothes canyou make limited or unlimi...,How many clothes can the clothes company make ...,What are clothing stores that sell inexpensive...,0.913168,0.028470,0.884699
3,How is GST beneficial for a common man?,How is GST beneficial for the country? How wou...,What exactly is GST? It's consequences and ben...,0.516488,0.317523,0.198966
4,What painting/drawing describes your personali...,What painting or drawing describes your person...,How does art influence feelings and make an im...,0.984973,0.268788,0.716185


In [13]:
print(f"Average Pos/Neg Diff: {df['pos_neg_diff'].mean()}")
print(f"Median Pos/Neg Diff: {df['pos_neg_diff'].median()}")

Average Pos/Neg Diff: 0.41804373264312744
Median Pos/Neg Diff: 0.42097043991088867
